In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gc
import seaborn as sns
import matplotlib.pyplot as plt

In [0]:
import tensorflow as tf

In [135]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [0]:
from keras.applications import VGG16

In [0]:
clf = VGG16(weights='imagenet',include_top=False,input_shape=(100, 100, 3))

In [138]:
clf.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 100, 100, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 100, 100, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 100, 100, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 50, 50, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 50, 50, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 50, 50, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 25, 25, 128)       0         
__________

In [0]:
import cv2 as cv

In [0]:
import os,shutil

In [145]:
for i in os.listdir('../dataset'):
  print(i)

train
sample_submissioin.csv
CNN_VGG.h5
Validation
test


In [24]:
for i in os.listdir('../dataset/train'):
  print(len([j for j in os.listdir('../dataset/train/'+i)]))

3752
1248
8203
2935
656
5301
6186
784
1248
4848
7936
5083
8835
4560
683


In [0]:
for i in range(15):
  os.mkdir('../dataset/Validation/'+str(i))

In [26]:
for i in os.listdir('../dataset/train/'):
  all_files = [j for j in os.listdir('../dataset/train/'+i)]
  print(i)
  for j in range(int(len(all_files)*.15)):
    src,dst = '../dataset/train/'+i+'/'+all_files[j],'../dataset/Validation/'+i+'/'+all_files[j]
    shutil.move(src,dst)
  
    

8
7
3
13
14
9
1
0
11
12
2
6
4
10
5


In [27]:
for i in range(15):
  print(len([j for j in os.listdir('../dataset/train/'+str(i))]),'  -  ',len([j for j in os.listdir('../dataset/Validation/'+str(i))]))

667   -   117
5259   -   927
6746   -   1190
6973   -   1230
7510   -   1325
581   -   102
4321   -   762
1061   -   187
3190   -   562
4506   -   795
3876   -   684
1061   -   187
4121   -   727
2495   -   440
558   -   98


In [0]:
train_dir = '../dataset/train'
validation_dir = '../dataset/Validation'

In [0]:
resize_size = (100,100)
def resize(image):
  return cv.resize(image,resize_size)

In [0]:
def normalize(image):
  return image/255

In [0]:
import pandas as pd
import numpy as np
from keras import models,layers
import os,shutil
from keras.preprocessing.image import ImageDataGenerator 

In [0]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   featurewise_center=False,  # set input mean to 0 over the dataset
                                    samplewise_center=False,  # set each sample mean to 0
                                    featurewise_std_normalization=False,  # divide inputs by std of the dataset
                                    samplewise_std_normalization=False,  # divide each input by its std
                                    zca_whitening=False,  # apply ZCA whitening
                                    rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
                                    zoom_range = 0.1, # Randomly zoom image 
                                    width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
                                    height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
                                    horizontal_flip=False,  # randomly flip images
                                    vertical_flip=False)
val_datagen = ImageDataGenerator(rescale=1./255)

In [68]:
train_generator = train_datagen.flow_from_directory(
    train_dir,target_size=(100, 100),batch_size=270,
    class_mode='categorical')

Found 52925 images belonging to 15 classes.


In [69]:
val_generator = val_datagen.flow_from_directory(
    validation_dir,target_size=(100, 100),batch_size=30,
    class_mode='categorical')

Found 9333 images belonging to 15 classes.


In [70]:
train_generator.class_indices,val_generator.class_indices

({'0': 0,
  '1': 1,
  '10': 2,
  '11': 3,
  '12': 4,
  '13': 5,
  '14': 6,
  '2': 7,
  '3': 8,
  '4': 9,
  '5': 10,
  '6': 11,
  '7': 12,
  '8': 13,
  '9': 14},
 {'0': 0,
  '1': 1,
  '10': 2,
  '11': 3,
  '12': 4,
  '13': 5,
  '14': 6,
  '2': 7,
  '3': 8,
  '4': 9,
  '5': 10,
  '6': 11,
  '7': 12,
  '8': 13,
  '9': 14})

In [0]:
from keras import optimizers,metrics

In [0]:
model = models.Sequential()

In [0]:
model.add(clf)

In [0]:
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dropout(0.4))
model.add(layers.Dense(15, activation='softmax'))

In [151]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 3, 3, 512)         14714688  
_________________________________________________________________
flatten_4 (Flatten)          (None, 4608)              0         
_________________________________________________________________
dense_7 (Dense)              (None, 512)               2359808   
_________________________________________________________________
dropout_4 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 15)                7695      
Total params: 17,082,191
Trainable params: 17,082,191
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.trainable = True
set_trainable = False
for layer in model.layers:
  if layer.name == 'block5_conv1':
    set_trainable = True
  if set_trainable:
    layer.trainable = True
  else:
    layer.trainable = False

In [0]:
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(),
              metrics=['acc'])

In [85]:
history = model.fit_generator(train_generator,steps_per_epoch=80,epochs=10,
                            validation_data=val_generator,validation_steps=363)

Epoch 1/10
80/80 [==============================] - 118s 1s/step - loss: 2.5916 - acc: 0.2353 - val_loss: 2.1030 - val_acc: 0.2907
Epoch 2/10
80/80 [==============================] - 107s 1s/step - loss: 2.1045 - acc: 0.3006 - val_loss: 1.9892 - val_acc: 0.3365
Epoch 3/10
80/80 [==============================] - 106s 1s/step - loss: 2.0067 - acc: 0.3356 - val_loss: 1.8663 - val_acc: 0.3855
Epoch 4/10
80/80 [==============================] - 108s 1s/step - loss: 1.9696 - acc: 0.3458 - val_loss: 1.8322 - val_acc: 0.3997
Epoch 5/10
11/80 [===>..........................] - ETA: 37s - loss: 1.9448 - acc: 0.355680/80 [==============================] - 107s 1s/step - loss: 1.9307 - acc: 0.3578 - val_loss: 1.8598 - val_acc: 0.3903
Epoch 6/10
80/80 [==============================] - 107s 1s/step - loss: 1.9107 - acc: 0.3660 - val_loss: 1.8045 - val_acc: 0.4154
Epoch 7/10
80/80 [==============================] - 108s 1s/step - loss: 1.8690 - acc: 0.3745 - val_loss: 1.7421 - val_acc: 0.4264
Epoch

In [0]:
# model.save('../dataset/CNN_VGG.h5')

In [0]:
# from google.colab import files
# files.download('../dataset/CNN_VGG.h5')

In [0]:
# cv.imread('../dataset/train/0/n02854926_10038_0.jpg').shape

In [0]:
import numpy as np

In [114]:
for j in os.listdir('../dataset/train/'):
  for i in os.listdir('../dataset/train/'+j):
    print(cv.resize(cv.imread('../dataset/train/'+j+'/'+i),(100,100)).shape,int(j))
    break

(100, 100, 3) 8
(100, 100, 3) 7
(100, 100, 3) 3
(100, 100, 3) 13
(100, 100, 3) 14
(100, 100, 3) 9
(100, 100, 3) 1
(100, 100, 3) 0
(100, 100, 3) 11
(100, 100, 3) 12
(100, 100, 3) 2
(100, 100, 3) 6
(100, 100, 3) 4
(100, 100, 3) 10
(100, 100, 3) 5


In [0]:
train = np.array([(cv.resize(cv.imread('../dataset/train/'+j+'/'+i),(100,100)) for i in os.listdir('../dataset/train/'+j)) for j in os.listdir('../dataset/train/')])

In [0]:
x_train, y_train = [], []
for i in os.listdir('../dataset/train'):
  for j in os.listdir('../dataset/train/'+i):
    x_train.append(cv.resize(cv.imread('../dataset/train/'+i+'/'+j),(100,100)))
    y_train.append(int(i))

In [0]:
x_train, y_train = np.array(x_train),np.array(y_train)

In [153]:
x_train.shape,y_train.shape

((52925, 100, 100, 3), (52925,))

In [0]:
test = np.array([cv.resize(cv.imread('../dataset/test/'+i),(100,100)) for i in os.listdir('../dataset/test')])

In [154]:
test.shape

(21273, 100, 100, 3)

In [0]:
result = model.predict_classes(test)

In [0]:
from collections import Counter

In [98]:
Counter(result)

Counter({0: 3,
         1: 715,
         2: 625,
         3: 361,
         4: 1460,
         5: 889,
         7: 4288,
         8: 6793,
         9: 2486,
         10: 118,
         11: 935,
         12: 71,
         13: 487,
         14: 2042})

In [0]:
def mapp(x):
    if x == 10:
        return 2
    elif x==11:
        return 3
    elif x==12:
        return 4
    elif x==13:
        return 5
    elif x==14:
        return 6
    elif x==2:
        return 7
    elif x==3:
        return 8
    elif x==4:
        return 9
    elif x==5:
        return 10
    elif x==6:
        return 11
    elif x==7:
        return 12
    elif x==8:
        return 13
    elif x==9:
        return 14
    else:
        return x
    

In [0]:
y_pred = []
for i in result:
    y_pred.append(mapp(i))

In [101]:
Counter(y_pred)

Counter({0: 3,
         1: 715,
         2: 118,
         3: 935,
         4: 71,
         5: 487,
         6: 2042,
         7: 625,
         8: 361,
         9: 1460,
         10: 889,
         12: 4288,
         13: 6793,
         14: 2486})

In [102]:
for i in os.listdir('../dataset/train'):
  print(i,'  ',len([j for j in os.listdir('../dataset/train/'+i)]))

8    3190
7    1061
3    6973
13    2495
14    558
9    4506
1    5259
0    667
11    1061
12    4121
2    6746
6    4321
4    7510
10    3876
5    581
